<table class="ee-notebook-buttons" align="left">
    <td><a target="_parent"  href="https://github.com/gee-community/geemap/tree/master/tutorials/ImageCollection/04_mapping_over_image_collection.ipynb"><img width=32px src="https://www.tensorflow.org/images/GitHub-Mark-32px.png" /> View source on GitHub</a></td>
    <td><a target="_parent"  href="https://nbviewer.jupyter.org/github/gee-community/geemap/blob/master/tutorials/ImageCollection/04_mapping_over_image_collection.ipynb"><img width=26px src="https://upload.wikimedia.org/wikipedia/commons/thumb/3/38/Jupyter_logo.svg/883px-Jupyter_logo.svg.png" />Notebook Viewer</a></td>
    <td><a target="_parent"  href="https://colab.research.google.com/github/gee-community/geemap/blob/master/tutorials/ImageCollection/04_mapping_over_image_collection.ipynb"><img width=26px src="https://www.tensorflow.org/images/colab_logo_32px.png" /> Run in Google Colab</a></td>
</table>

# Mapping over an ImageCollection
To apply a function to every `Image` in an `ImageCollection` use `imageCollection.map()`. The only argument to `map()` is a function which takes one parameter: an `ee.Image`. For example, the following code adds a timestamp band to every image in the collection:

## Install Earth Engine API and geemap
Install the [Earth Engine Python API](https://developers.google.com/earth-engine/python_install) and [geemap](https://github.com/gee-community/geemap). The **geemap** Python package is built upon the [ipyleaflet](https://github.com/jupyter-widgets/ipyleaflet) and [folium](https://github.com/python-visualization/folium) packages and implements several methods for interacting with Earth Engine data layers, such as `Map.addLayer()`, `Map.setCenter()`, and `Map.centerObject()`.
The following script checks if the geemap package has been installed. If not, it will install geemap, which automatically installs its [dependencies](https://github.com/gee-community/geemap#dependencies), including earthengine-api, folium, and ipyleaflet.

**Important note**: A key difference between folium and ipyleaflet is that ipyleaflet is built upon ipywidgets and allows bidirectional communication between the front-end and the backend enabling the use of the map to capture user input, while folium is meant for displaying static data only ([source](https://blog.jupyter.org/interactive-gis-in-jupyter-with-ipyleaflet-52f9657fa7a)). Note that [Google Colab](https://colab.research.google.com/) currently does not support ipyleaflet ([source](https://github.com/googlecolab/colabtools/issues/60#issuecomment-596225619)). Therefore, if you are using geemap with Google Colab, you should use [`import geemap.foliumap`](https://github.com/gee-community/geemap/blob/master/geemap/foliumap.py). If you are using geemap with [binder](https://mybinder.org/) or a local Jupyter notebook server, you can use [`import geemap`](https://github.com/gee-community/geemap/blob/master/geemap/geemap.py), which provides more functionalities for capturing user input (e.g., mouse-clicking and moving).

In [ ]:
# Installs geemap package
import subprocess

try:
    import geemap
except ImportError:
    print("geemap package not installed. Installing ...")
    subprocess.check_call(["python", "-m", "pip", "install", "geemap"])

# Checks whether this notebook is running on Google Colab
try:
    import google.colab
    import geemap.foliumap as emap
except:
    import geemap as emap

# Authenticates and initializes Earth Engine
import ee

try:
    ee.Initialize()
except Exception as e:
    ee.Authenticate()
    ee.Initialize()

## Create an interactive map 
The default basemap is `Google Satellite`. [Additional basemaps](https://github.com/gee-community/geemap/blob/master/geemap/geemap.py#L13) can be added using the `Map.add_basemap()` function. 

In [ ]:
Map = geemap.Map(center=[40, -100], zoom=4)
Map.add_basemap("ROADMAP")  # Add Google Map
Map

## Add Earth Engine Python script 

In [ ]:
# This function adds a band representing the image timestamp.
def addTime(image):
    return image.addBands(image.metadata("system:time_start"))


# Load a Landsat 8 collection for a single path-row.
collection = (
    ee.ImageCollection("LANDSAT/LC08/C01/T1_TOA")
    .filter(ee.Filter.eq("WRS_PATH", 44))
    .filter(ee.Filter.eq("WRS_ROW", 34))
)

first = collection.map(addTime).first()

# Map the function over the collection and display the result.
# print(collection.map(addTime).getInfo())
print(first.bandNames().getInfo())

For one-line function, you can also use a [lambda function](https://www.w3schools.com/python/python_lambda.asp), which is a small anonymous function. 

In [ ]:
# Load a Landsat 8 collection for a single path-row.
collection = (
    ee.ImageCollection("LANDSAT/LC08/C01/T1_TOA")
    .filter(ee.Filter.eq("WRS_PATH", 44))
    .filter(ee.Filter.eq("WRS_ROW", 34))
)

first = collection.map(
    lambda image: image.addBands(image.metadata("system:time_start"))
).first()

# Map the function over the collection and display the result.
# print(collection.map(addTime).getInfo())
print(first.bandNames().getInfo())

Note that in the predefined function, the `metadata()` method is used to create a new `Image` from the value of a property. As discussed in the [Reducing](https://developers.google.com/earth-engine/ic_reducing) and [Compositing](https://developers.google.com/earth-engine/ic_composite_mosaic) sections, having that time band is useful for the linear modeling of change and for making composites.

The mapped function is limited in the operations it can perform. Specifically, it can’t modify variables outside the function; it can’t print anything; it can’t use JavaScript ‘if’ or ‘for’ statements. However, you can use `ee.Algorithms.If()` to perform conditional operations in a mapped function. For example:

In [ ]:
# This function uses a conditional statement to return the image if
# the solar elevation > 40 degrees.  Otherwise it returns a zero image.
def conditional(image):
    return ee.Algorithms.If(
        ee.Number(image.get("SUN_ELEVATION")).gt(40), image, ee.Image(0)
    )


# Load a Landsat 8 collection for a single path-row.
collection = (
    ee.ImageCollection("LANDSAT/LC08/C01/T1_TOA")
    .filter(ee.Filter.eq("WRS_PATH", 44))
    .filter(ee.Filter.eq("WRS_ROW", 34))
)

# Load a Landsat 8 collection for a single path-row.
collection = (
    ee.ImageCollection("LANDSAT/LC8_L1T_TOA")
    .filter(ee.Filter.eq("WRS_PATH", 44))
    .filter(ee.Filter.eq("WRS_ROW", 34))
)

# Map the function over the collection, convert to a List and print the result.
# print('Expand this to see the result: ', collection.map(conditional).getInfo())

Inspect the list of images in the output ImageCollection and note that the when the condition evaluated by the `If()` algorithm is true, the output contains a constant image. Although this demonstrates a server-side conditional function (learn more about client vs. server in Earth Engine on [this page](https://developers.google.com/earth-engine/client_server)), avoid `If()` in general and use filters instead.

In [ ]:
# a function for calculating NDVI for Landsat 8 imagery
def calNDVI(image):
    return image.normalizedDifference(["B5", "B4"])


point = ee.Geometry.Point(-83.93, 35.85)

# select three Landsat 8 imagery in the Knoxville area with the least cloud coverage
collection = (
    ee.ImageCollection("LANDSAT/LC8_L1T_TOA")
    .filterBounds(point)
    .sort("CLOUD_COVER")
    .limit(3)
)

# calculate NDVI for each image using the map function
ndvi_images = collection.map(calNDVI)

vizParams = {"bands": ["B5", "B4", "B3"], "min": 0, "max": 0.4}

Map = geemap.Map(center=[40, -100], zoom=4)


Map.centerObject(point, 8)

for i in range(0, 3):
    landsat = ee.Image(collection.toList(3).get(i))
    ndvi = ee.Image(ndvi_images.toList(3).get(i))

    Map.addLayer(landsat, vizParams, "Landsat image " + str(i + 1))
    Map.addLayer(ndvi, {"palette": ["red", "green"]}, "NDVI image " + str(i + 1))

Map.addLayerControl()
Map

Use the lambda function 

In [ ]:
point = ee.Geometry.Point(-83.93, 35.85)

# select three Landsat 8 imagery in the Knoxville area with the least cloud coverage
collection = (
    ee.ImageCollection("LANDSAT/LC8_L1T_TOA")
    .filterBounds(point)
    .sort("CLOUD_COVER")
    .limit(3)
)

# calculate NDVI for each image using the map function
ndvi_images = collection.map(lambda image: image.normalizedDifference(["B5", "B4"]))

vizParams = {"bands": ["B5", "B4", "B3"], "min": 0, "max": 0.4}

Map = geemap.Map(center=[40, -100], zoom=4)


Map.centerObject(point, 8)

for i in range(0, 3):
    landsat = ee.Image(collection.toList(3).get(i))
    ndvi = ee.Image(ndvi_images.toList(3).get(i))

    Map.addLayer(landsat, vizParams, "Landsat image " + str(i + 1))
    Map.addLayer(ndvi, {"palette": ["red", "green"]}, "NDVI image " + str(i + 1))

Map.addLayerControl()
Map